In [1]:
import numpy as np
import pandas as pd


In [2]:
df = pd.read_csv('../data/Spotify_exo2.csv')
df.head()

,mode,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,21st century classical,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.361600,-31.514333,0.040567,75.336500,0.103783,27.833333,6
1,1,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.131000,-16.854000,0.076817,120.285667,0.221750,52.500000,5
2,1,8-bit,0.762000,0.712000,1.151770e+05,0.818000,0.876000,0.126000,-9.180000,0.047000,133.444000,0.975000,48.000000,7
3,1,[],0.651417,0.529093,2.328809e+05,0.419146,0.205309,0.218696,-12.288965,0.107872,112.857352,0.513604,20.859882,7
4,1,a cappella,0.676557,0.538961,1.906285e+05,0.316434,0.003003,0.172254,-12.479387,0.082851,112.110362,0.448249,45.820071,7


In [15]:
df.info()
print(df.describe())

print(f'\nNaN Values:\n{df.isnull().sum()}')
print(np.sum(df.isnull().sum()))
df = df[df.genres != '[]']

columns = df.columns.values
target = "popularity"
features = np.delete(columns, np.argwhere(columns==target))

X = df[features]
y = df[target]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2972 entries, 0 to 2972
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   mode              2972 non-null   int64  
 1   genres            2972 non-null   object 
 2   acousticness      2972 non-null   float64
 3   danceability      2972 non-null   float64
 4   duration_ms       2972 non-null   float64
 5   energy            2972 non-null   float64
 6   instrumentalness  2972 non-null   float64
 7   liveness          2972 non-null   float64
 8   loudness          2972 non-null   float64
 9   speechiness       2972 non-null   float64
 10  tempo             2972 non-null   float64
 11  valence           2972 non-null   float64
 12  popularity        2972 non-null   float64
 13  key               2972 non-null   int64  
dtypes: float64(11), int64(2), object(1)
memory usage: 348.3+ KB
              mode  acousticness  danceability   duration_ms       energy  \

### Handling with genres

In [4]:
genres = np.array(X.genres)
print(len(genres))
print(len(np.unique(genres)))

2972
2972


In [5]:
# from sklearn.feature_extraction.text import CountVectorizer
#
# text = ["i love nlp. nlp is so cool"]
# vectorizer = CountVectorizer()
#
# # tokenize and build vocab
# vectorizer.fit(text)
# print(vectorizer.vocabulary_)
# # encode document
#
# vector = vectorizer.transform(text)
# # summarize encoded vector
#
# print(vector.shape) # Output: (1, 5)
# print(vector.toarray())


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

lexique = list()

for genre in genres:
    for mot in genre.split(sep=' '):
        lexique.append(mot)

lexique = np.array(lexique)
unique = np.unique(lexique)

vectorizer = CountVectorizer(strip_accents='unicode')

# tokenize and build vocab
vectorizer.fit(unique)
# print(vectorizer.vocabulary_)

test = vectorizer.transform(genres)

cool = test.toarray()

In [8]:
def getKeysByValue(dictOfElements, valueToFind):
    listOfKeys = list()
    listOfItems = dictOfElements.items()
    for item  in listOfItems:
        if item[1] == valueToFind:
            listOfKeys.append(item[0])
    return  listOfKeys


col_names = list()

for i in range(len(vectorizer.vocabulary_)):
    col_names.append(getKeysByValue(vectorizer.vocabulary_, i)[0])


df_genres = pd.DataFrame(cool, columns=col_names)

In [9]:
res_comp = list()

for k in range(len(genres)):
    mot_mystere = np.array(df_genres.loc[k])
    indexes = np.where(mot_mystere != 0)[0]
    trouve = ""
    for i in indexes:
        trouve += getKeysByValue(vectorizer.vocabulary_, i)[0]+" "
    trouve = trouve[:-1]

    res_comp.append((trouve, genres[k]))

In [10]:
wrong_reconstitution = [(res[0], res[1]) for res in res_comp if res[0] != res[1]]

In [11]:
df_genres.insert(0, "genres", genres)

In [12]:
print(df_genres.shape)
print(df.shape)

(2972, 1633)
(2972, 14)


## Merge 2 dataframes

In [ ]:
dfinal = df.merge(df_genres, on="genres", how = 'inner')